# Settings

In [1]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rnd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
%matplotlib inline

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load data

In [2]:
import pickle
import boto3
from io import BytesIO

In [3]:
s3 = boto3.resource('s3')
data_subsets = ['train', 'val']
data = {}

for name in data_subsets:
    with BytesIO() as files:
        path = "omniglot_images/" +name+ ".pickle"
        s3.Bucket("research-paper-omniglot-data").download_fileobj(path, files)
        files.seek(0)    # move back to the beginning after writing
        (X,c) = pickle.load(files)
        data[name] = X

In [4]:
def create_train_data(size, s='train'):
    #get train data and shape
    X=data[s]
    n_classes, n_examples, w, h = X.shape
    
    #initialize 2 empty arrays for the input size in a list
    pairs=[np.zeros((size, h, w,1)) for i in range(2)]
    
    #initialize vector for the targets
    targets=np.zeros((size,1))
    
    for x in range(size):
        #randomly sample one class (character)
        category = rnd.choice(n_classes,1,replace=False)
        #randomly sample one example from class (1-20 characters)
        idx_1 = rnd.randint(0, n_examples)
        pairs[0][x,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        #randomly sample again one example from class and add last class with modulo
        # ..to ensure not same class pairs are created
        idx_2 = (idx_1 + rnd.randint(0, n_examples)) % n_examples
        #pick images of different class for 1st half and same class for 2nd half
        if x >= size // 2:
            category_2 = category
            targets[x] = 1
        else: 
        #add a random number to the category modulo n classes to ensure 2nd image has
        # ..different category
            idx_2 = rnd.randint(0, n_examples) 
            category_2 = (category + rnd.randint(1,n_classes)) % n_classes
            targets[x] = 0
        pairs[1][x,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
        
    return pairs, targets

In [5]:
train_set, train_labels = create_train_data(10000)
#val_set, val_labels = create_train_data(10000)

# Create graph

In [5]:
def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = rnd.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)

In [6]:
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=rnd.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

In [7]:
input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv2D(64,(10,10),activation='relu',input_shape=input_shape,
                   kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(7,7),activation='relu',
                   kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(256,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))

#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#layer to merge two encoded inputs with the l1 distance between them
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(0.00006)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()

38951745

# Training

In [8]:
def get_batch(batch_size,s="train"):
    """Create batch of n pairs, half same class, half different class"""
    X=data[s]
    n_classes, n_examples, w, h = X.shape

    #randomly sample several classes to use in the batch
    categories = rnd.choice(n_classes,size=(batch_size,),replace=False)
    #initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, h, w,1)) for i in range(2)]
    #initialize vector for the targets, and make one half of it '1's, so 2nd half of batch has same class
    targets=np.zeros((batch_size,))
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = rnd.randint(0, n_examples)
        pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        idx_2 = rnd.randint(0, n_examples)
        #pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            #add a random number to the category modulo n classes to ensure 2nd image has
            # ..different category
            category_2 = (category + rnd.randint(1,n_classes)) % n_classes
        pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h,1)

        return(pairs, targets)

In [9]:
def generate_oneshot_set(N, s='val'):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    X=data[s]
    n_classes, n_examples, w, h = X.shape
    indices = rnd.randint(0,n_examples,size=(N,))
    categories = rnd.choice(range(n_classes),size=(N,),replace=False)            
    true_category = categories[0]
    ex1, ex2 = rnd.choice(n_examples,replace=False,size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
    support_set = X[categories,indices,:,:]
    support_set[0,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, w, h,1)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets

In [10]:
def prediction(probs, labels):
    probs_rounded = np.round_(probs)
    output = np.equal(probs_rounded, labels)
    accuracy = np.mean(output.astype(int))
    
    return(accuracy)

This is just as fast as the Tensorflow implementation

In [13]:
#Training loop
print("!")
evaluate_every = 50 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 90000
N_way = 20 # how many classes for testing one-shot tasks>
n_val = 250 #how mahy one-shot tasks to validate on?

print("training")
for i in range(1, n_iter):
    #batch_x1, batch_x2, batch_y = shuffle(train_set[0],train_set[1], train_labels, n_samples = batch_size)
    #loss=siamese_net.train_on_batch([batch_x1, batch_x2],batch_y)
    batch_x, batch_y = get_batch(batch_size)
    loss=siamese_net.train_on_batch(batch_x,batch_y)
    if i % evaluate_every == 0:
        train_x, train_y = get_batch(10)
        val_x, val_y = get_batch(10, s='val')
        prob_train = siamese_net.predict(train_x)
        prob_val = siamese_net.predict(val_x)
        print(prob_train)
        acc_train = prediction(prob_train, train_y)
        acc_val = prediction(prob_val, val_y)
        
        print('acc train:', acc_train)
        print('acc val:', acc_val)
        print('batch iteration:', i)
        print('loss:', loss)
        #print("evaluating")
        #n_correct = 0
        #for j in range(n_val):
            #pairs, labels = generate_oneshot_set(N_way, s='val')
            #probs = siamese_net.predict(pairs)
            #if np.argmax(probs) == np.argmax(labels):
                    #n_correct+=1
        #percent_correct = (n_correct / n_val)
        #print('Validation accuracy:', percent_correct)

!
training
[[4.1997666e-04]
 [6.1986721e-01]
 [6.1986721e-01]
 [6.1986721e-01]
 [6.1986721e-01]
 [6.1986721e-01]
 [6.1986721e-01]
 [6.1986721e-01]
 [6.1986721e-01]
 [6.1986721e-01]]
acc train: 0.5
acc val: 0.5
batch iteration: 50
loss: 1.2691745
[[0.01054339]
 [0.6191726 ]
 [0.6191726 ]
 [0.6191726 ]
 [0.6191726 ]
 [0.6191726 ]
 [0.6191726 ]
 [0.6191726 ]
 [0.6191726 ]
 [0.6191726 ]]
acc train: 0.5
acc val: 0.5
batch iteration: 100
loss: 1.0633879
[[0.00675219]
 [0.6184767 ]
 [0.6184767 ]
 [0.6184767 ]
 [0.6184767 ]
 [0.6184767 ]
 [0.6184767 ]
 [0.6184767 ]
 [0.6184767 ]
 [0.6184767 ]]
acc train: 0.5
acc val: 0.5
batch iteration: 150
loss: 0.9358061
[[1.1086735e-04]
 [6.1778516e-01]
 [6.1778516e-01]
 [6.1778516e-01]
 [6.1778516e-01]
 [6.1778516e-01]
 [6.1778516e-01]
 [6.1778516e-01]
 [6.1778516e-01]
 [6.1778516e-01]]
acc train: 0.5
acc val: 0.5
batch iteration: 200
loss: 0.85136485
[[1.6615999e-05]
 [6.1709505e-01]
 [6.1709505e-01]
 [6.1709505e-01]
 [6.1709505e-01]
 [6.1709505e-01]
 [6

KeyboardInterrupt: 

In [18]:
np.round_([0.1,0.3,0.51])

array([0., 0., 1.])

In [16]:
batch, label = get_batch(10)

In [20]:
batch[0][9]

array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])